In [37]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Функция для генерации случайных дат
def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

# Функция для генерации вероятностей удержания
def generate_retention_probabilities(n_days):
    retention_prob = np.zeros(n_days)
    retention_prob[:7] = 0.5     # 50% для первых 7 дней
    retention_prob[7:21] = 0.3   # 30% для 2-3 недели
    retention_prob[21:42] = 0.1  # 10% для 4-6 недели
    retention_prob[42:] = 0.05   # 5% после 6 недель
    return retention_prob

# Генерация данных для таблицы customers
def generate_customers(n_days, avg_customers_per_day):
    total_customers = n_days * avg_customers_per_day
    customer_ids = np.arange(1, total_customers + 1)
    signup_dates = np.array([datetime(2020, 1, 1) + timedelta(days=i // avg_customers_per_day) for i in range(total_customers)])
    regions = np.random.choice(['Москва', 'Тюмень', 'Новосибирск', 'Санкт-Петербург'], total_customers)
    
    return pd.DataFrame({
        'customer_id': customer_ids,
        'region': regions,
        'signup_date': signup_dates
    })

# Генерация данных для таблицы products
def generate_products(n):
    product_ids = range(1, n + 1)
    categories = ['телефоны', 'наушники', 'аксессуары', 'аккумуляторы']
    prices = np.random.uniform(50, 2000, n)

    return pd.DataFrame({
        'product_id': product_ids,
        'category': np.random.choice(categories, n),
        'price': prices.round(2)
    })

# Генерация данных для таблицы orders
def generate_orders(customers, products, n_days):
    retention_prob = generate_retention_probabilities(n_days)
    orders = []
    platforms = ['mobile', 'desktop']

    for day in range(n_days):
        order_date = datetime(2020, 1, 1) + timedelta(days=day)
        cohort_customers = customers[customers['signup_date'] <= order_date].copy()
        cohort_customers['days_since_signup'] = (order_date - cohort_customers['signup_date']).dt.days
        cohort_customers['order_probability'] = retention_prob[np.clip(cohort_customers['days_since_signup'], 0, n_days - 1)]

        rand_vals = np.random.rand(len(cohort_customers))
        cohort_customers = cohort_customers[rand_vals < cohort_customers['order_probability']]

        if not cohort_customers.empty:
            for _, row in cohort_customers.iterrows():
                num_orders = np.random.choice([1, 2, 3, 4, 5], p=[0.5, 0.3, 0.1, 0.07, 0.03])
                for _ in range(num_orders):
                    orders.append({
                        'order_id': len(orders) + 1,
                        'customer_id': row['customer_id'],
                        'order_date': order_date.date(),  # Только дата
                        'order_amount': round(np.random.uniform(50, 5000), 2),
                        'platform': random.choice(platforms),
                        'category': random.choice(products['category'].values)
                    })

    return pd.DataFrame(orders)

# Параметры
n_days = 365
avg_customers_per_day = 10
n_products = 500

# Генерация данных
customers = generate_customers(n_days, avg_customers_per_day)
products = generate_products(n_products)
orders = generate_orders(customers, products, n_days)

# Сохранение в CSV
customers.to_csv('customers.csv', index=False)
products.to_csv('products.csv', index=False)
orders.to_csv('orders.csv', index=False)
orders['order_date'] = pd.to_datetime(orders['order_date'], errors='coerce')

print("Данные успешно сгенерированы и сохранены в CSV.")

Данные успешно сгенерированы и сохранены в CSV.


In [38]:
orders.dtypes

order_id                 int64
customer_id              int64
order_date      datetime64[ns]
order_amount           float64
platform                object
category                object
dtype: object

In [42]:
orders

,order_id,customer_id,order_date,order_amount,platform,category
0,1,4,2020-01-01,52.38,mobile,аксессуары
1,2,5,2020-01-01,3321.79,mobile,аксессуары
2,3,5,2020-01-01,3088.47,mobile,наушники
3,4,1,2020-01-02,1147.16,mobile,телефоны
4,5,1,2020-01-02,1045.20,mobile,аккумуляторы
...,...,...,...,...,...,...
110881,110882,3647,2020-12-30,3086.40,mobile,телефоны
110882,110883,3648,2020-12-30,1796.68,mobile,телефоны
110883,110884,3648,2020-12-30,1945.32,desktop,аккумуляторы
110884,110885,3649,2020-12-30,3186.56,desktop,аккумуляторы


In [43]:
orders[orders['customer_id'] == 3]

,order_id,customer_id,order_date,order_amount,platform,category
8,9,3,2020-01-02,4200.69,mobile,наушники
9,10,3,2020-01-02,1812.47,mobile,наушники
10,11,3,2020-01-02,2979.24,mobile,аксессуары
11,12,3,2020-01-02,3093.81,desktop,телефоны
30,31,3,2020-01-03,2108.25,desktop,телефоны
31,32,3,2020-01-03,574.92,desktop,аккумуляторы
55,56,3,2020-01-04,3513.42,mobile,аксессуары
56,57,3,2020-01-04,849.65,mobile,телефоны
90,91,3,2020-01-05,542.96,mobile,телефоны
149,150,3,2020-01-06,4784.79,mobile,аккумуляторы


In [46]:
first_order_df = orders.groupby('customer_id', as_index=False).agg({'order_date' : 'min'}).rename(columns={'order_date': 'first_order_date'})
first_order_df

,customer_id,first_order_date
0,1,2020-01-02
1,2,2020-01-05
2,3,2020-01-02
3,4,2020-01-01
4,5,2020-01-01
...,...,...
3634,3645,2020-12-30
3635,3647,2020-12-30
3636,3648,2020-12-30
3637,3649,2020-12-30


In [86]:
first_order_df['cohort_month'] = first_order_df['first_order_date'].dt.to_period('M').dt.to_timestamp()
first_order_df

,customer_id,first_order_date,cohort_month
0,1,2020-01-02,2020-01-01
1,2,2020-01-05,2020-01-01
2,3,2020-01-02,2020-01-01
3,4,2020-01-01,2020-01-01
4,5,2020-01-01,2020-01-01
...,...,...,...
3634,3645,2020-12-30,2020-12-01
3635,3647,2020-12-30,2020-12-01
3636,3648,2020-12-30,2020-12-01
3637,3649,2020-12-30,2020-12-01


In [60]:
orders = orders.merge(first_order_df, on='customer_id')

In [61]:
orders

,order_id,customer_id,order_date,order_amount,platform,category,first_order_date,cohort_month
0,1,4,2020-01-01,52.38,mobile,аксессуары,2020-01-01,2020-01-01
1,2,5,2020-01-01,3321.79,mobile,аксессуары,2020-01-01,2020-01-01
2,3,5,2020-01-01,3088.47,mobile,наушники,2020-01-01,2020-01-01
3,4,1,2020-01-02,1147.16,mobile,телефоны,2020-01-02,2020-01-01
4,5,1,2020-01-02,1045.20,mobile,аккумуляторы,2020-01-02,2020-01-01
...,...,...,...,...,...,...,...,...
110881,110882,3647,2020-12-30,3086.40,mobile,телефоны,2020-12-30,2020-12-01
110882,110883,3648,2020-12-30,1796.68,mobile,телефоны,2020-12-30,2020-12-01
110883,110884,3648,2020-12-30,1945.32,desktop,аккумуляторы,2020-12-30,2020-12-01
110884,110885,3649,2020-12-30,3186.56,desktop,аккумуляторы,2020-12-30,2020-12-01


In [74]:
orders['delta_days'] = (orders['order_date'] - orders['first_order_date']).dt.days
orders_first_10_days = orders[orders['delta_days'] < 10]
orders_first_10_days

,order_id,customer_id,order_date,order_amount,platform,category,first_order_date,cohort_month,delta_days
0,1,4,2020-01-01,52.38,mobile,аксессуары,2020-01-01,2020-01-01,0
1,2,5,2020-01-01,3321.79,mobile,аксессуары,2020-01-01,2020-01-01,0
2,3,5,2020-01-01,3088.47,mobile,наушники,2020-01-01,2020-01-01,0
3,4,1,2020-01-02,1147.16,mobile,телефоны,2020-01-02,2020-01-01,0
4,5,1,2020-01-02,1045.20,mobile,аккумуляторы,2020-01-02,2020-01-01,0
...,...,...,...,...,...,...,...,...,...
110881,110882,3647,2020-12-30,3086.40,mobile,телефоны,2020-12-30,2020-12-01,0
110882,110883,3648,2020-12-30,1796.68,mobile,телефоны,2020-12-30,2020-12-01,0
110883,110884,3648,2020-12-30,1945.32,desktop,аккумуляторы,2020-12-30,2020-12-01,0
110884,110885,3649,2020-12-30,3186.56,desktop,аккумуляторы,2020-12-30,2020-12-01,0


In [79]:
total_customer_amount_df =  orders_first_10_days[['customer_id' , 'order_amount', 'cohort_month']].groupby(['customer_id', 'cohort_month'], as_index=False).agg({'order_amount' : 'sum'}).rename(columns={'order_amount' : 'total_customer_amount'})
total_customer_amount_df

,customer_id,cohort_month,total_customer_amount
0,1,2020-01-01,27905.71
1,2,2020-01-01,11697.79
2,3,2020-01-01,35534.27
3,4,2020-01-01,6420.84
4,5,2020-01-01,21122.42
...,...,...,...
3634,3645,2020-12-01,4162.84
3635,3647,2020-12-01,3086.40
3636,3648,2020-12-01,3742.00
3637,3649,2020-12-01,3186.56


In [83]:
cohort_month_df = total_customer_amount_df.groupby(['cohort_month'], as_index=False).agg({'total_customer_amount' : 'mean', 'customer_id' : pd.Series.nunique})
cohort_month_df

,cohort_month,total_customer_amount,customer_id
0,2020-01-01,21552.414040,297
1,2020-02-01,21534.940990,293
2,2020-03-01,21212.648127,315
3,2020-04-01,21044.088136,295
4,2020-05-01,21095.944268,314
5,2020-06-01,21961.615651,292
6,2020-07-01,22141.698147,313
7,2020-08-01,21353.551812,309
8,2020-09-01,20444.081433,300
9,2020-10-01,21135.066032,310
